In [17]:
# Description of the imported dataset
# The AP test score have been processed in SAS.
# Merged from STU_ATTR_TBL, STU_CRS_TBL, AP score.
# Selected target population (cohort, US high school, beginners)
# Only keep the first attempt on the course

## Load libraries
install.packages("pacman") # pacman helps load several packages at once

pacman::p_load("haven", # loading .sas7bdat file
               "tidyverse", "dplyr", "data.table", "psych") # data wrangling/descriptive stats

package 'pacman' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\feng8\AppData\Local\Temp\Rtmpkhv1o2\downloaded_packages


In [20]:
# Import full dataset
iubfull <- read_sas("X:/BAR/Projects/SEISMIC/MeasurementGroup/AP project/Data/Derived/unique_crs.sas7bdat")
names(iubfull) = tolower(names(iubfull))
names(iubfull)

[1] "prsn_univ_id"                  "cohort_term_cd"               
 [3] "cohort_term_desc"              "new_student_category"         
 [5] "ethnicity"                     "urm_flag"                     
 [7] "gender"                        "first_generation"             
 [9] "pell_eligibility"              "high_school_gpa"              
[11] "sat_drvd_composite_score"      "sat_drvd_composite_score_2019"
[13] "sat_erws_score"                "sat_mss_scr"                  
[15] "attended_us_high_school"       "mathprof"                     
[17] "engprof"                       "apchemsr"                     
[19] "apchemyr"                      "apbiolsr"                     
[21] "apbiolyr"                      "apphysmecsr"                  
[23] "apphysmecyr"                   "apphysemsr"                   
[25] "apphysemyr"                    "crs_type"                     
[27] "crs_subj_dept_cd"              "crs_catlg_nbr"                
[29] "crs_nm"                        "crs_ofcl_grd_nbr"             
[31] "crs_ofcl_grd_cd"               "crs_repeat_cd"                
[33] "acad_term_cd"                  "gpao"                         
[35] "acad_unt_tkn_nbr"              "cls_instr_nm"                 
[37] "crs_cmpnt_cd"                  "cls_nbr"                      
[39] "term_code"                     "term_end_units"               
[41] "crs_term_cd"                   "tme_frm_chrt"                 
[43] "aptaker_biol"                  "aptaker_chem"                 
[45] "aptaker_physmec"               "aptaker_physem"               
[47] "summerflg"

In [4]:
#####################################################
# Student level 
#####################################################
stu_attr <- iubfull %>%
  # Rename variables
  mutate(st_id = prsn_univ_id) %>%
  mutate(firstgen = recode(first_generation, "Y" = 1, "N" = 0)) %>%
  mutate(ethniccode = ethnicity) %>%
  mutate(ethniccode_cat = recode(urm_flag, "White" = 0, "URM" = 1, "Asian" = 2, "Other" = 0)) %>%
  mutate(female = recode(gender, "Female" = 1, "Male" = 0)) %>%
  mutate(lowincomflag = recode(pell_eligibility, "Y" = 1, "N" = 0)) %>%
  mutate(transfer = ifelse(new_student_category == "Transfer", 1, 0)) %>%
  mutate(international = ifelse(ethnicity == "NR-Alien", 1, 0)) %>%
  mutate(us_hs = ifelse(attended_us_high_school == "Y", 1, 0)) %>%
  mutate(cohort = substr(cohort_term_desc, 6, 9)) %>%
  mutate(englsr = sat_erws_score) %>%
  mutate(mathsr = sat_mss_scr) %>%
  mutate(hsgpa = high_school_gpa) %>%
  # Select common-named variables 
  select(st_id, firstgen:hsgpa) %>%
  # Keep unique ID
  distinct()
stu_attr$gender <- stu_attr$female
  
names(stu_attr)
head(stu_attr)
# write.csv(stu_attr, file = "X:/BAR/Projects/SEISMIC/MeasurementGroup/AP project/Data/Derived/AP_processed/stu_attr.csv", row.names = FALSE)  

[1] "st_id"          "firstgen"       "ethniccode"     "ethniccode_cat"
 [5] "female"         "lowincomflag"   "transfer"       "international" 
 [9] "us_hs"          "cohort"         "englsr"         "mathsr"        
[13] "hsgpa"          "gender"

st_id,firstgen,ethniccode,ethniccode_cat,female,lowincomflag,transfer,international,us_hs,cohort,englsr,mathsr,hsgpa,gender
0002212679,0,White,0,1,1,0,0,1,2017,590,650,4.00,1
0002343169,0,White,0,0,0,0,0,1,2014,680,660,4.00,0
0002457687,0,White,0,1,0,0,0,1,2018,660,590,4.00,1
0002468699,0,Asian,2,0,0,0,0,1,2013,770,780,4.00,0
0002491719,0,White,0,0,1,0,0,1,2013,670,610,2.84,0
0002513309,0,White,0,1,1,0,0,1,2013,610,670,3.82,1


In [5]:
#####################################################
# Course level 
#####################################################
stu_crs <- iubfull %>%
  mutate(st_id = prsn_univ_id) %>%
  mutate(crs_sbj = crs_subj_dept_cd) %>%
  mutate(crs_catalog = crs_catlg_nbr) %>%
  mutate(crs_name = crs_nm) %>%
  mutate(numgrade = crs_ofcl_grd_nbr) %>%
  mutate(numgrade_w = ifelse(crs_ofcl_grd_cd == "W", 1, 0)) %>%
  mutate(crs_retake = ifelse(crs_repeat_cd == "REPT", 1, 0)) %>%
  mutate(crs_term = acad_term_cd) %>%
  mutate(summer_crs = ifelse(substr(crs_term, 4, 1) == "5", 1, 0)) %>%
  mutate(enrl_from_cohort = substr(as.character(tme_frm_chrt/3-1/3), 1, 4)) %>%
  # mutate(begin_term_cum_gpa = ) %>%
  mutate(crs_credits = acad_unt_tkn_nbr) %>%
  mutate(instructor_name = cls_instr_nm) %>%
  mutate(crs_component = crs_cmpnt_cd) %>%
  mutate(class_number = cls_nbr) %>%
  # mutate(current_major = ) %>%
  # Select common-named variables used in analysis
  select(st_id, c(crs_type, crs_sbj:class_number, gpao))

names(stu_crs)  
head(stu_crs)

# write.csv(stu_crs, file = "X:/BAR/Projects/SEISMIC/MeasurementGroup/AP project/Data/Derived/AP_processed/stu_crs.csv", row.names = FALSE)  
  

[1] "st_id"            "crs_type"         "crs_sbj"          "crs_catalog"     
 [5] "crs_name"         "numgrade"         "numgrade_w"       "crs_retake"      
 [9] "crs_term"         "summer_crs"       "enrl_from_cohort" "crs_credits"     
[13] "instructor_name"  "crs_component"    "class_number"     "gpao"

st_id,crs_type,crs_sbj,crs_catalog,crs_name,numgrade,numgrade_w,crs_retake,crs_term,summer_crs,enrl_from_cohort,crs_credits,instructor_name,crs_component,class_number,gpao
0002212679,ENRL,BIOL,112,BIOL-L112,3.0,0,0,4182,0,0.33,4,"Dunn,Megan Marie",LEC,1630,3.051
0002343169,ACP,BIOL,112,BIOL-L112,4.0,0,0,4128,0,NA,3,,,1710,NA
0002343169,ACP,PHYS,221,PHYS-P221,3.7,0,1,4138,0,NA,5,,,35167,NA
0002343169,ENRL,PHYS,221,PHYS-P221,3.0,0,0,4148,0,0,5,"Kaufman,Lisa Jean",LEC,10566,3.478
0002457687,ACP,BIOL,112,BIOL-L112,3.3,0,0,4168,0,NA,3,,,19334,NA
0002468699,ENRL,BIOL,112,BIOL-L112,3.7,0,0,4138,0,0,3,"Berndtson,Amy",LEC,6085,3.966


In [12]:
# Get the ID for students who took honor version of introductory course
biol_honor <- stu_crs %>%
  filter(crs_name == 'BIOL-H111') %>%
  select(st_id, crs_name) %>%
  distinct()

# Select biol course 1&2
biol_crs_long <- stu_crs %>% 
  filter(crs_name %in% c('BIOL-L112', 'BIOL-L211')) %>%
  select(st_id, c(crs_name, crs_term, summer_crs, enrl_from_cohort, numgrade)) %>%
  group_by(st_id, crs_name) %>%
  arrange(crs_term, .by_group = TRUE) %>%
  distinct(st_id, crs_name, .keep_all = TRUE)
  

# Transpose from long to wide
biol_crs_wide <- reshape(as.data.frame(biol_crs_long), idvar = "st_id", timevar = "crs_name", direction = "wide")

colnames(biol_crs_wide) <- c("st_id", "crs_term_1", "summer_crs_1", "enrl_from_cohort_1", "numgrade_1",
                             "crs_term_2", "summer_crs_2", "enrl_from_cohort_2", "numgrade_2")

biol_crs <- biol_crs_wide %>%
  left_join(biol_honor, by = "st_id") %>%
  mutate(honor_1 = ifelse(crs_name %in% 'BIOL-H111', 1, 0)) %>%
  select(-one_of("crs_name"))

names(biol_crs)
head(biol_crs)

####################Chemistry##########################
# Get the ID for students who took honor version of introductory course
chem_honor <- stu_crs %>%
  filter(crs_name == 'CHEM-S117') %>%
  select(st_id, crs_name) %>%
  distinct()

# Select chem course 1&2
chem_crs_long <- stu_crs %>% 
  filter(crs_name %in% c('CHEM-C117', 'CHEM-C341')) %>%
  select(st_id, c(crs_name, crs_term, summer_crs, enrl_from_cohort, numgrade)) %>%
  group_by(st_id, crs_name) %>%
  arrange(crs_term, .by_group = TRUE) %>%
  distinct(st_id, crs_name, .keep_all = TRUE)

# Transpose from long to wide
chem_crs_wide <- reshape(as.data.frame(chem_crs_long), idvar = "st_id", timevar = "crs_name", direction = "wide")

colnames(chem_crs_wide) <- c("st_id", "crs_term_1", "summer_crs_1", "enrl_from_cohort_1", "numgrade_1",
                             "crs_term_2", "summer_crs_2", "enrl_from_cohort_2", "numgrade_2")

chem_crs <- chem_crs_wide %>%
  left_join(chem_honor, by = "st_id") %>%
  mutate(honor_1 = ifelse(crs_name %in% 'CHEM-S117', 1, 0)) %>%
  select(-one_of("crs_name"))

####################Physics##########################
# Get the ID for students who took honor version of introductory course
phys_honor <- stu_crs %>%
  filter(crs_name == 'PHYS-H221') %>%
  select(st_id, crs_name) %>%
  distinct()

# Select phys course 1&2
phys_crs_long <- stu_crs %>% 
  filter(crs_name %in% c('PHYS-P221', 'PHYS-P222')) %>%
  select(st_id, c(crs_name, crs_term, summer_crs, enrl_from_cohort, numgrade)) %>%
  group_by(st_id, crs_name) %>%
  arrange(crs_term, .by_group = TRUE) %>%
  distinct(st_id, crs_name, .keep_all = TRUE)

# Transpose from long to wide
phys_crs_wide <- reshape(as.data.frame(phys_crs_long), idvar = "st_id", timevar = "crs_name", direction = "wide")

colnames(phys_crs_wide) <- c("st_id", "crs_term_1", "summer_crs_1", "enrl_from_cohort_1", "numgrade_1",
                             "crs_term_2", "summer_crs_2", "enrl_from_cohort_2", "numgrade_2")

phys_crs <- phys_crs_wide %>%
  left_join(phys_honor, by = "st_id") %>%
  mutate(honor_1 = ifelse(crs_name %in% 'PHYS-H221', 1, 0)) %>%
  select(-one_of("crs_name"))

Warning message:
"Column `st_id` has different attributes on LHS and RHS of join"

[1] "st_id"              "crs_term_1"         "summer_crs_1"      
 [4] "enrl_from_cohort_1" "numgrade_1"         "crs_term_2"        
 [7] "summer_crs_2"       "enrl_from_cohort_2" "numgrade_2"        
[10] "honor_1"

st_id,crs_term_1,summer_crs_1,enrl_from_cohort_1,numgrade_1,crs_term_2,summer_crs_2,enrl_from_cohort_2,numgrade_2,honor_1
0002212679,4182,0,0.33,3.0,NA,NA,NA,NA,0
0002343169,4128,0,NA,4.0,NA,NA,NA,NA,0
0002457687,4168,0,NA,3.3,NA,NA,NA,NA,0
0002468699,4138,0,0,3.7,NA,NA,NA,NA,0
0002491719,4142,0,0.33,0.0,NA,NA,NA,NA,0
0002513309,4148,0,1,1.7,NA,NA,NA,NA,0


Warning message:
"Column `st_id` has different attributes on LHS and RHS of join"Warning message:
"Column `st_id` has different attributes on LHS and RHS of join"

In [14]:
#####################################################
# AP by subjects 
#####################################################
ap_biol <- iubfull %>%
  mutate(st_id = prsn_univ_id) %>%
  mutate(aptaker = aptaker_biol) %>%
  mutate(apscore = apbiolsr) %>%
  mutate(apyear = apbiolyr) %>% 
  mutate(eligible_to_skip = ifelse(is.na(apbiolsr), 0,
                            ifelse(apbiolsr %in% c(4,5), 1, 0))) %>%
  mutate(tookcourse = ifelse(crs_nm == 'BIOL-L112' 
                              & crs_type %in% c('ENRL', 'ACP', 'XFR'), 1, 0)) %>%
  select(st_id, aptaker:tookcourse) %>%
  group_by(st_id) %>%
  summarize_at(vars(-group_cols()), max)

names(ap_biol)
head(ap_biol)


ap_chem <- iubfull %>%
  mutate(st_id = prsn_univ_id) %>%
  mutate(aptaker = aptaker_chem) %>%
  mutate(apscore = apchemsr) %>%
  mutate(apyear = apchemyr) %>% 
  mutate(eligible_to_skip = ifelse(is.na(apchemsr), 0,
                                   ifelse(apchemsr == 5, 1, 0))) %>%
  mutate(tookcourse = ifelse(crs_nm == 'CHEM-C117' 
                             & crs_type %in% c('ENRL', 'ACP', 'XFR'), 1, 0)) %>%
  select(st_id, aptaker:tookcourse) %>%
  group_by(st_id) %>%
  summarize_at(vars(-group_cols()), max)

ap_phys <- iubfull %>%
  mutate(st_id = prsn_univ_id) %>%
  mutate(aptaker = aptaker_physmec) %>%
  mutate(apscore = apphysmecsr) %>%
  mutate(apyear = apphysmecyr) %>% 
  mutate(eligible_to_skip = ifelse(is.na(apphysmecsr), 0,
                                   ifelse(apphysmecsr == 5, 1, 0))) %>%
  mutate(tookcourse = ifelse(crs_nm == 'PHYS-P221' 
                             & crs_type %in% c('ENRL', 'ACP', 'XFR'), 1, 0)) %>%
  select(st_id, aptaker:tookcourse) %>%
  group_by(st_id) %>%
  summarize_at(vars(-group_cols()), max)

[1] "st_id"            "aptaker"          "apscore"          "apyear"          
[5] "eligible_to_skip" "tookcourse"

st_id,aptaker,apscore,apyear,eligible_to_skip,tookcourse
0002212679,0,NA,NA,0,1
0002343169,0,NA,NA,0,1
0002457687,0,NA,NA,0,1
0002468699,0,NA,NA,0,1
0002491719,0,NA,NA,0,1
0002513309,0,NA,NA,0,1


In [15]:
#####################################################
# Full vars 
#####################################################

# Biology
biol <- stu_attr %>%
  full_join(ap_biol, by = "st_id") %>%
  right_join(biol_crs, by = "st_id") %>%
  mutate(skipped_course = ifelse(eligible_to_skip == 1
                                 & tookcourse == 0, 1, 0)) %>%
  filter(!is.na(crs_term_2)) %>%
  filter(honor_1 == 0) %>%
  filter(apyear >= 2013 | is.na(apyear)) %>%
  mutate(discipline = "BIO")

table(biol$eligible_to_skip)
table(biol$skipped_course)

names(biol)
head(biol)

# Chemistry
chem <- stu_attr %>%
  full_join(ap_chem, by = "st_id") %>%
  right_join(chem_crs, by = "st_id") %>%
  mutate(skipped_course = ifelse(eligible_to_skip == 1
                                 & tookcourse == 0, 1, 0)) %>%
  filter(!is.na(crs_term_2)) %>%
  filter(honor_1 == 0) %>%
  filter(apyear >= 2014 | is.na(apyear)) %>%
  mutate(discipline = "CHEM")

table(chem$eligible_to_skip)
table(chem$skipped_course)

names(chem)
head(chem)

# Physics
phys <- stu_attr %>%
  full_join(ap_phys, by = "st_id") %>%
  right_join(phys_crs, by = "st_id") %>%
  mutate(skipped_course = ifelse(eligible_to_skip == 1
                                 & tookcourse == 0, 1, 0)) %>%
  filter(!is.na(crs_term_2)) %>%
  filter(honor_1 == 0) %>%
  filter(apyear >= 2015 | is.na(apyear)) %>%
  mutate(discipline = "PHYS")

table(phys$eligible_to_skip)
table(phys$skipped_course)

names(phys)
head(phys)

Warning message:
"Column `st_id` has different attributes on LHS and RHS of join"


   0    1 
2676  599 


   0    1 
3128  147 

[1] "st_id"              "firstgen"           "ethniccode"        
 [4] "ethniccode_cat"     "female"             "lowincomflag"      
 [7] "transfer"           "international"      "us_hs"             
[10] "cohort"             "englsr"             "mathsr"            
[13] "hsgpa"              "gender"             "aptaker"           
[16] "apscore"            "apyear"             "eligible_to_skip"  
[19] "tookcourse"         "crs_term_1"         "summer_crs_1"      
[22] "enrl_from_cohort_1" "numgrade_1"         "crs_term_2"        
[25] "summer_crs_2"       "enrl_from_cohort_2" "numgrade_2"        
[28] "honor_1"            "skipped_course"     "discipline"

st_id,firstgen,ethniccode,ethniccode_cat,female,lowincomflag,transfer,international,us_hs,cohort,...,summer_crs_1,enrl_from_cohort_1,numgrade_1,crs_term_2,summer_crs_2,enrl_from_cohort_2,numgrade_2,honor_1,skipped_course,discipline
0002607686,0,Asian,2,1,0,0,0,1,2013,...,0,1,2.3,4152,0,1.33,0.7,0,0,BIO
0002609268,1,White,0,0,0,0,0,1,2013,...,0,0,4.0,4152,0,1.33,3.3,0,0,BIO
0002615928,1,White,0,1,1,0,0,1,2013,...,0,1.33,4.0,4158,0,2,4.0,0,0,BIO
0002674044,0,White,0,1,1,0,0,1,2014,...,NA,NA,NA,4152,0,0.33,3.7,0,1,BIO
0002743869,0,White,0,1,1,0,0,1,2014,...,0,1,3.7,4162,0,1.33,3.0,0,0,BIO
0002744813,0,White,0,1,0,0,0,1,2014,...,0,0.33,2.3,4162,0,1.33,NA,0,0,BIO


...

Warning message:
"Column `st_id` has different attributes on LHS and RHS of join"


  0   1 
531  34 


  0   1 
535  30 

[1] "st_id"              "firstgen"           "ethniccode"        
 [4] "ethniccode_cat"     "female"             "lowincomflag"      
 [7] "transfer"           "international"      "us_hs"             
[10] "cohort"             "englsr"             "mathsr"            
[13] "hsgpa"              "gender"             "aptaker"           
[16] "apscore"            "apyear"             "eligible_to_skip"  
[19] "tookcourse"         "crs_term_1"         "summer_crs_1"      
[22] "enrl_from_cohort_1" "numgrade_1"         "crs_term_2"        
[25] "summer_crs_2"       "enrl_from_cohort_2" "numgrade_2"        
[28] "honor_1"            "skipped_course"     "discipline"

st_id,firstgen,ethniccode,ethniccode_cat,female,lowincomflag,transfer,international,us_hs,cohort,...,summer_crs_1,enrl_from_cohort_1,numgrade_1,crs_term_2,summer_crs_2,enrl_from_cohort_2,numgrade_2,honor_1,skipped_course,discipline
0002823500,0,White,0,0,0,0,0,1,2013,...,0,1.33,3.3,4158,0,2,3.0,0,0,PHYS
0002824262,0,Asian,2,1,0,0,0,1,2013,...,0,1,3.7,4152,0,1.33,4.0,0,0,PHYS
0002827924,0,Hispanic/Latino,1,0,0,0,0,1,2013,...,0,1.33,3.0,4158,0,2,3.3,0,0,PHYS
0002829454,0,Asian,2,1,0,0,0,1,2013,...,0,1,2.7,4152,0,1.33,3.7,0,0,PHYS
0002830080,0,White,0,0,0,0,0,1,2013,...,0,1,3.7,4152,0,1.33,4.0,0,0,PHYS
0002838001,0,White,0,1,0,0,0,1,2013,...,0,0.33,4.0,4148,0,1,2.3,0,0,PHYS


In [16]:
#####################################################
# Stacked data
#####################################################
df_clean <- rbind(biol, chem, phys)
df_clean <- df_clean %>%
  rename_at(vars(ends_with("_2")),
            ~ (str_replace(., "_2", "")))

names(df_clean)
head(df_clean)

# write.csv(df_clean, file = "X:/BAR/Projects/SEISMIC/MeasurementGroup/AP project/Data/Derived/AP_processed/df_clean.csv", row.names = FALSE)  


[1] "st_id"              "firstgen"           "ethniccode"        
 [4] "ethniccode_cat"     "female"             "lowincomflag"      
 [7] "transfer"           "international"      "us_hs"             
[10] "cohort"             "englsr"             "mathsr"            
[13] "hsgpa"              "gender"             "aptaker"           
[16] "apscore"            "apyear"             "eligible_to_skip"  
[19] "tookcourse"         "crs_term_1"         "summer_crs_1"      
[22] "enrl_from_cohort_1" "numgrade_1"         "crs_term"          
[25] "summer_crs"         "enrl_from_cohort"   "numgrade"          
[28] "honor_1"            "skipped_course"     "discipline"

st_id,firstgen,ethniccode,ethniccode_cat,female,lowincomflag,transfer,international,us_hs,cohort,...,summer_crs_1,enrl_from_cohort_1,numgrade_1,crs_term,summer_crs,enrl_from_cohort,numgrade,honor_1,skipped_course,discipline
0002607686,0,Asian,2,1,0,0,0,1,2013,...,0,1,2.3,4152,0,1.33,0.7,0,0,BIO
0002609268,1,White,0,0,0,0,0,1,2013,...,0,0,4.0,4152,0,1.33,3.3,0,0,BIO
0002615928,1,White,0,1,1,0,0,1,2013,...,0,1.33,4.0,4158,0,2,4.0,0,0,BIO
0002674044,0,White,0,1,1,0,0,1,2014,...,NA,NA,NA,4152,0,0.33,3.7,0,1,BIO
0002743869,0,White,0,1,1,0,0,1,2014,...,0,1,3.7,4162,0,1.33,3.0,0,0,BIO
0002744813,0,White,0,1,0,0,0,1,2014,...,0,0.33,2.3,4162,0,1.33,NA,0,0,BIO
